In [75]:
import numpy as np 
import pandas as pd

In [76]:
mic_coords = pd.read_excel("data/mic_coords_final.xlsx", index_col=0)
distances = pd.read_excel("data/miscellaneous_positions.xlsx", index_col=0)

# remove NaNs
distances = distances.drop(index=[12, 13])
print(distances)

def get_mic_coords(name, df, x, y, z):
    filtered_df = df[df['Mic_Index'] == name]
    coordinates = filtered_df[[x, y, z]].to_numpy()
    return coordinates[0]

#coords = get_mic_coords("B4", mic_coords, "X", "Y", "Z")

items = ["basson_low", "basson_high", "camera_1", "camera_2"]

position_coords_default = []
for name in distances["point 2"]:
    coord = get_mic_coords(name, mic_coords, "X", "Y", "Z")
    position_coords_default.append(coord)

position_coords_optimized = []
for name in distances["point 2"]:
    coord = get_mic_coords(name, mic_coords, "X_optimized", "Y_optimized", "Z_optimized")
    position_coords_optimized.append(coord)



                        point 1 point 2  distance
0   basson lower mounting point      B3     0.505
1   basson lower mounting point     B10     0.740
2   basson lower mounting point      B6     0.684
3        basson bell top middle     C12     0.484
4        basson bell top middle     C16     0.560
5        basson bell top middle     C17     0.493
6   camera pos 1 (lense center)      B5     0.826
7   camera pos 1 (lense center)      C6     0.837
8   camera pos 1 (lense center)     A10     1.490
9                  camera pos 2      A7     0.470
10                 camera pos 2     B13     0.835
11                 camera pos 2     C14     0.926


In [82]:
def triangulate_point(list_coords, df_distances, start):
    P1 = list_coords[start]
    P2 = list_coords[start+1]
    P3 = list_coords[start+2]

    d1 = df_distances["distance"][start]
    d2 = df_distances["distance"][start+1]
    d3 = df_distances["distance"][start+2]

    # Build the system of linear equations
    A = 2 * np.array([
        [P2[0] - P1[0], P2[1] - P1[1], P2[2] - P1[2]],
        [P3[0] - P1[0], P3[1] - P1[1], P3[2] - P1[2]]
        ])

    b = np.array([
        d1**2 - d2**2 + np.dot(P2, P2) - np.dot(P1, P1),
        d1**2 - d3**2 + np.dot(P3, P3) - np.dot(P1, P1)
        ])

    # Solve for x, y, z using least squares
    xyz = np.linalg.lstsq(A, b, rcond=None)[0]

    # Compute z-coordinate explicitly (if needed)
    z_squared = d1**2 - np.dot(xyz - P1, xyz - P1)
    print("z_squared = ", z_squared) # complex means bad :(
    z = np.sqrt(z_squared) if z_squared >= 0 else None

    print("Coordinates of the unknown point:")
    print(f"x: {xyz[0]}, y: {xyz[1]}, z: ±{z}")


start_points = [0, 3, 6, 9]

print("DEFAULT CALCULATED VALUES")
print("")
for start, item in zip(start_points, items):
    print(item)
    triangulate_point(position_coords_default, distances, start)
    print("")


print("")
print("OPTIMIZED VALUES")
for start, item in zip(start_points, items):
    print(item)
    triangulate_point(position_coords_optimized, distances, start)
    print("")



DEFAULT CALCULATED VALUES

basson_low
z_squared =  -1.6732682979236642
Coordinates of the unknown point:
x: 5.551115123125783e-17, y: 1.028945827439887, z: ±None

basson_high
z_squared =  0.12382873765795283
Coordinates of the unknown point:
x: 0.519369940308797, y: 0.0, z: ±0.3518930770247588

camera_1
z_squared =  0.1774531162692473
Coordinates of the unknown point:
x: 0.6892581157127653, y: 0.7052751790482733, z: ±0.4212518442324583

camera_2
z_squared =  -2.3338110273767843
Coordinates of the unknown point:
x: -0.24319943832747654, y: 1.1540472506670563, z: ±None


OPTIMIZED VALUES
basson_low
z_squared =  -1.819024328491085
Coordinates of the unknown point:
x: 0.00223031678506469, y: 1.0053779533482348, z: ±None

basson_high
z_squared =  0.12382873765795283
Coordinates of the unknown point:
x: 0.5257270504102248, y: -0.05682104576017928, z: ±0.3518930770247588

camera_1
z_squared =  0.132899785070402
Coordinates of the unknown point:
x: 0.6986960438917849, y: 0.7016823714541304, z:

In [78]:
from _mic_position_functions import calculate_distance

_ = calculate_distance(mic_coords, "B10", "B3", x_col="X_optimized", y_col="Y_optimized", z_col="Z_optimized")
_ = calculate_distance(mic_coords, "B6", "B3", x_col="X_optimized", y_col="Y_optimized", z_col="Z_optimized")
_ = calculate_distance(mic_coords, "B10", "B6", x_col="X_optimized", y_col="Y_optimized", z_col="Z_optimized")

0.703 = distance B10 to B3
0.737 = distance B6 to B3
0.713 = distance B10 to B6
